<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загружаем-данные" data-toc-modified-id="Загружаем-данные-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загружаем данные</a></span></li><li><span><a href="#func" data-toc-modified-id="func-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>func</a></span></li><li><span><a href="#features" data-toc-modified-id="features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>features</a></span></li><li><span><a href="#fit" data-toc-modified-id="fit-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>fit</a></span></li><li><span><a href="#baseline-cv" data-toc-modified-id="baseline-cv-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>baseline cv</a></span></li><li><span><a href="#sklearn-api" data-toc-modified-id="sklearn-api-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><del>sklearn api</del></a></span></li><li><span><a href="#predict" data-toc-modified-id="predict-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>predict</a></span></li><li><span><a href="#save" data-toc-modified-id="save-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>save</a></span></li></ul></div>

In [406]:
import pandas as pd

from xgboost import XGBRegressor
import xgboost as xgb

# Загружаем данные

In [448]:
train_pool = pd.read_csv('train.csv', encoding='utf-8').sort_values('query_id')
test_pool = pd.read_csv('test.csv', encoding='utf-8').sort_values('query_id')

# func

Напишем функции, генерирующие простые признаки основанные на пересечении триграмм между запросом и названием организации

In [449]:
import metrics as m
import numpy as np
def ndcg(y_true, y_pred, groups):
    return np.nanmean([m.ndcg_score(np.array(y_true)[np.array(groups) == g], 
                                 np.array(y_pred)[np.array(groups) == g]) for g in set(groups)])

In [450]:
def get_trigrams(string):
    string = '^^' + string + '$$'
    trigrams = set()
    trigrams_count = 0
    
    for i in range(len(string) - 2):
        trigrams.add(string[i:i+3])
        trigrams_count += 1
        
    return trigrams, trigrams_count

def common_trigrams_factors(query, org_name):
    query_trigrams, query_trigrams_count = get_trigrams(query)
    org_name_trigrams, org_name_trigrams_count = get_trigrams(org_name)

    factors = [float(len(query_trigrams.intersection(org_name_trigrams)))]

    factors.append(0. if query_trigrams_count == 0. else 0.1 + factors[0] / query_trigrams_count)
    factors.append(0. if org_name_trigrams_count == 0. else 0.1 + factors[0] / org_name_trigrams_count)    
    
    return factors

In [451]:
def get_bigrams(string):
    string = '^^' + string + '$$'
    bigrams = set()
    bigrams_count = 0
    
    for i in range(len(string) - 1):
        bigrams.add(string[i:i+2])
        bigrams_count += 1
        
    return bigrams, bigrams_count

def common_bigrams_factors(query, org_name):
    query_bigrams, query_bigrams_count = get_bigrams(query)
    org_name_bigrams, org_name_bigrams_count = get_bigrams(org_name)

    factors = [float(len(query_bigrams.intersection(org_name_bigrams)))]

    factors.append(0. if query_bigrams_count == 0. else 0.1 + factors[0] / query_bigrams_count)
    factors.append(0. if org_name_bigrams_count == 0. else 0.1 + factors[0] / org_name_bigrams_count)    
    
    return factors

In [452]:
def get_grams(string, n=3):
    string = list('^^') + string + list('$$')
    bigrams = set()
    bigrams_count = 0
    
    for i in range(len(string) - n+1):
        bigrams.add('_'.join(string[i:i+n]))
        bigrams_count += 1
        
    return bigrams, bigrams_count

def common_grams_factors(query, org_name, n):
    query_bigrams, query_bigrams_count = get_grams(query, n)
    org_name_bigrams, org_name_bigrams_count = get_grams(org_name, n)

    factors = [float(len(query_bigrams.intersection(org_name_bigrams)))]

    factors.append(0. if query_bigrams_count == 0. else 0.1 + factors[0] / query_bigrams_count)
    factors.append(0. if org_name_bigrams_count == 0. else 0.1 + factors[0] / org_name_bigrams_count)    
    
    return factors

# features

Посчитаем данные факторы для каждого файла

In [453]:
train_group_sizes = train_pool.query_id.value_counts()

train_pool = train_pool.join(train_group_sizes.to_frame('group_sizes'), on='query_id')

In [454]:
test_group_sizes = test_pool.query_id.value_counts()

test_pool = test_pool.join(test_group_sizes.to_frame('group_sizes'), on='query_id')

In [455]:
train_pool.head()

,query_id,query,region,org_name,org_id,window_center,window_size,relevance,group_sizes
0,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Суд Жовтневого району міста Дніпропетровськ,1021049127,"34.613119,48.506531","0.025928,0.017380",0.00,20
19,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Дніпровський районний суд,1171413301,"34.613119,48.506531","0.025928,0.017380",0.61,20
18,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Заводський суд,1218937827,"34.613119,48.506531","0.025928,0.017380",0.61,20
17,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Індустріальний районний суд м. Дніпропетровська,1082076884,"34.613119,48.506531","0.025928,0.017380",0.00,20
15,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Дніпропетровський апеляційний адміністративний...,1586735257,"34.613119,48.506531","0.025928,0.017380",0.00,20


In [468]:
# def calc_trigram_factors(row):
#     return pd.Series(common_trigrams_factors(list(row.query), list(row.org_name)))
# def calc_bigram_factors(row):
#     return pd.Series(common_bigrams_factors(row.query, row.org_name))
def calc_gram_factors(row, n):
    return pd.Series(common_grams_factors(list(row.query), list(row.org_name), n))
def calc_word_gram_factors(row, n):
    return pd.Series(common_grams_factors(row.query.split(), row.org_name.split(), n))
    
train_factors = pd.concat((
#                            train_pool.apply(calc_gram_factors, n=1, axis=1), 
                           train_pool.apply(calc_gram_factors, n=2, axis=1), 
                           train_pool.apply(calc_gram_factors, n=3, axis=1), 
                           train_pool.apply(calc_word_gram_factors, n=1, axis=1), 
                           train_pool.apply(calc_word_gram_factors, n=2, axis=1), 
#                            train_pool.apply(calc_gram_factors, n=4, axis=1),
#                           train_pool.window_center.str.split(',').apply(pd.Series),
#                           train_pool.window_size.str.split(',').apply(pd.Series)
                          ), axis=1)
test_factors = pd.concat((
#                            test_pool.apply(calc_gram_factors, n=1, axis=1),
                           test_pool.apply(calc_gram_factors, n=2, axis=1),
                           test_pool.apply(calc_gram_factors, n=3, axis=1),
                           test_pool.apply(calc_word_gram_factors, n=1, axis=1),
                           test_pool.apply(calc_word_gram_factors, n=2, axis=1),
#                            test_pool.apply(calc_gram_factors, n=4, axis=1),
#                           test_pool.window_center.str.split(',').apply(pd.Series),
#                           test_pool.window_size.str.split(',').apply(pd.Series)
                        ), axis=1)

In [469]:
train_factors

,0,1,2,0,1,2,0,1,2,0,1,2
0,17.0,0.323684,0.469565,9.0,0.220000,0.300000,2.0,0.281818,0.322222,2.0,0.300000,0.350000
19,14.0,0.284211,0.600000,7.0,0.193333,0.359259,2.0,0.281818,0.385714,2.0,0.300000,0.433333
18,10.0,0.231579,0.688235,5.0,0.166667,0.412500,2.0,0.281818,0.433333,2.0,0.300000,0.500000
17,20.0,0.363158,0.500000,12.0,0.260000,0.344898,2.0,0.281818,0.322222,2.0,0.300000,0.350000
15,19.0,0.350000,0.458491,12.0,0.260000,0.330769,2.0,0.281818,0.350000,2.0,0.300000,0.385714
14,20.0,0.363158,0.500000,14.0,0.286667,0.385714,2.0,0.281818,0.350000,2.0,0.300000,0.385714
13,26.0,0.442105,0.665217,18.0,0.340000,0.500000,2.0,0.281818,0.350000,2.0,0.300000,0.385714
12,11.0,0.244737,0.523077,4.0,0.153333,0.260000,2.0,0.281818,0.385714,2.0,0.300000,0.433333
11,21.0,0.376316,0.520000,12.0,0.260000,0.344898,2.0,0.281818,0.350000,2.0,0.300000,0.385714
10,8.0,0.205263,0.375862,4.0,0.153333,0.242857,2.0,0.281818,0.385714,2.0,0.300000,0.433333


# fit

Обучаем модель

In [470]:
clf = XGBRegressor(n_estimators=100)
clf.fit(train_factors.values, train_pool[['relevance']].values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# baseline cv

In [471]:
from sklearn.model_selection import cross_val_predict, GroupKFold, cross_val_score

In [472]:
clf = XGBRegressor(n_estimators=100)
baseline_pred = cross_val_predict(
    clf,
    train_factors.values,
    train_pool[['relevance']].values,
    groups=train_pool.query_id.values,
    cv=5)
ndcg(train_pool.relevance.values, baseline_pred, train_pool.query_id.values)

0.7248211594405837

In [473]:
# from sklearn.linear_model import LinearRegression
# clf = LinearRegression()
# lr_pred = cross_val_predict(
#     clf,
#     X=train_factors.values,
#     y=train_pool[['relevance']].values,
#     groups=train_pool.query_id.values,
#     cv=5)
# ndcg(train_pool.relevance.values, lr_pred, train_pool.query_id.values)

In [474]:
# from sklearn.svm import SVR
# clf = SVR()
# svr_pred = cross_val_predict(
#     clf,
#     X=train_factors.values,
#     y=train_pool[['relevance']].values,
#     groups=train_pool.query_id.values,
#     cv=5)
# ndcg(train_pool.relevance.values, svr_pred, train_pool.query_id.values)

# ~~sklearn api~~

In [476]:
kf = GroupKFold(4)

In [475]:
from sklearn.base import clone

def cross_val_predict_custom(train_pool=train_pool, train_factors=train_factors, cv_n=5, 
                             params={'objective': 'rank:pairwise', 'silent': True}, n_trees=50):
    kf = GroupKFold(cv_n)
    cv_preds = np.zeros(train_pool.shape[0])
    for tr_i, te_i in kf.split(train_factors.values, train_pool.relevance.values, train_pool.query_id.values):
        tr_i = sorted(tr_i)
        te_i = sorted(te_i)
        
        train_queries = train_pool.query_id.iloc[tr_i].drop_duplicates().values
        test_queries = train_pool.query_id.iloc[te_i].drop_duplicates().values
        
        train_dmatrix = xgb.DMatrix(train_factors.values[tr_i], train_pool.relevance.values[tr_i])
        train_dmatrix.set_group(train_group_sizes.loc[train_queries].values)

        test_dmatrix = xgb.DMatrix(train_factors.values[te_i])
        test_dmatrix.set_group(train_group_sizes.loc[test_queries].values)
        
        regr = xgb.train(params, train_dmatrix, n_trees)
        cv_preds[te_i] = regr.predict(test_dmatrix)
    return cv_preds, params

In [495]:
cv_p, params = cross_val_predict_custom(params={
                'objective': 'rank:pairwise', 
                'silent': True,
                'max_depth': 4,
                'colsample_bytree': colsample_bytree
    #     'colsample_bytree': 0.6000000000000001,
    #     'eta': 0.07500000000000001,
    #     'gamma': 0.8500000000000001,
            },n_trees= 50)
ndcg(train_pool.relevance.values, cv_p, train_pool.query_id.values)

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


0.7489986422667344

In [496]:
params

{'objective': 'rank:pairwise',
 'silent': True,
 'max_depth': 4,
 'colsample_bytree': 1.0}

In [478]:
p = {}
met = 0.7
for n_trees in [20, 30, 50, 80]:
    for max_depth in [2, 3, 4, 5]:
        for colsample_bytree in [1]:
#         np.linspace(0.6, 1, 5):
            cv_p, params = cross_val_predict_custom(params={
                'objective': 'rank:pairwise', 
                'silent': True,
                'max_depth': max_depth,
                'colsample_bytree': colsample_bytree
    #     'colsample_bytree': 0.6000000000000001,
    #     'eta': 0.07500000000000001,
    #     'gamma': 0.8500000000000001,
            },n_trees= n_trees)

            m2 = ndcg(train_pool.relevance.values, cv_p, train_pool.query_id.values)
            if m2 > met:
                p['n_trees'] = n_trees
                p['max_depth'] = max_depth
                p['colsample_bytree'] = colsample_bytree
                met = m2
                print(p, m2)

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 20, 'max_depth': 2, 'colsample_bytree': 1} 0.7452805472945973


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 20, 'max_depth': 3, 'colsample_bytree': 1} 0.7473062775261374


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


...

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 30, 'max_depth': 3, 'colsample_bytree': 1} 0.7506089909923365


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 30, 'max_depth': 4, 'colsample_bytree': 1} 0.7515525047713881


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: Run

In [480]:
for n_trees in [40, 60]:
    for max_depth in [2, 3, 4, 5]:
        for colsample_bytree in [1]:
#         np.linspace(0.6, 1, 5):
            cv_p, params = cross_val_predict_custom(params={
                'objective': 'rank:pairwise', 
                'silent': True,
                'max_depth': max_depth,
                'colsample_bytree': colsample_bytree
    #     'colsample_bytree': 0.6000000000000001,
    #     'eta': 0.07500000000000001,
    #     'gamma': 0.8500000000000001,
            },n_trees= n_trees)

            m2 = ndcg(train_pool.relevance.values, cv_p, train_pool.query_id.values)
            if m2 > met:
                p['n_trees'] = n_trees
                p['max_depth'] = max_depth
                p['colsample_bytree'] = colsample_bytree
                met = m2
                print(p, m2)

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 40, 'max_depth': 3, 'colsample_bytree': 1} 0.7519392293035174


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


In [481]:
for n_trees in [40, 60]:
    for max_depth in [2, 3, 4, 5]:
        for colsample_bytree in  np.linspace(0.6, 1, 5):
            cv_p, params = cross_val_predict_custom(params={
                'objective': 'rank:pairwise', 
                'silent': True,
                'max_depth': max_depth,
                'colsample_bytree': colsample_bytree
    #     'colsample_bytree': 0.6000000000000001,
    #     'eta': 0.07500000000000001,
    #     'gamma': 0.8500000000000001,
            },n_trees= n_trees)

            m2 = ndcg(train_pool.relevance.values, cv_p, train_pool.query_id.values)
            if m2 > met:
                p['n_trees'] = n_trees
                p['max_depth'] = max_depth
                p['colsample_bytree'] = colsample_bytree
                met = m2
                print(p, m2)

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 40, 'max_depth': 3, 'colsample_bytree': 0.7} 0.7524805727829385


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


{'n_trees': 40, 'max_depth': 3, 'colsample_bytree': 0.8} 0.7527020681417089


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: Run

{'n_trees': 60, 'max_depth': 3, 'colsample_bytree': 0.8} 0.7528436813752863


D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best
D:\Courses\shad2018\ML\competition2\metrics.py:147: Run

In [482]:
p

{'n_trees': 60, 'max_depth': 3, 'colsample_bytree': 0.8}

In [483]:
ndcg(train_pool.relevance.values, cv_p, train_pool.query_id.values)

0.7405008303297754

# predict

In [484]:
train_dmatrix = xgb.DMatrix(train_factors.values, train_pool.relevance)
train_dmatrix.set_group(train_group_sizes.values)

In [485]:
test_dmatrix = xgb.DMatrix(test_factors.values)
test_dmatrix.set_group(test_group_sizes.values)

In [497]:
clf = xgb.train(params, train_dmatrix, 50)
predictions = clf.predict(test_dmatrix)
test_pool['relevance'] = predictions

# save

Сортируем организации по предсказанной релевантности для каждой организации и записываем их в файл

In [498]:
test_pool.sort_values(['query_id', 'relevance'], ascending=[True, False])[['query_id', 'org_id']].to_csv('submission3.csv', index=None)

In [488]:
params

{'objective': 'rank:pairwise',
 'silent': True,
 'max_depth': 5,
 'colsample_bytree': 1.0}